# Introduction

Welcome to the **Few Shot Object Detection for TensorFlow Lite** Colab. Here, we demonstrate fine tuning of a SSD architecture (pre-trained on COCO) on very few examples of a *novel* class. We will then generate a (downloadable) TensorFlow Lite model for on-device inference.

**NOTE:** This Colab is meant for the few-shot detection use-case. To train a model on a large dataset, please follow the [TF2 training](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_training_and_evaluation.md#training) documentation and then [convert](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_on_mobile_tf2.md) the model to TensorFlow Lite.

# Set Up

## Imports

In [ ]:
!python --version
!python -m pip show tensorflow

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import io
import imageio
import glob
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

tfds = tf.data.Dataset

from object_detection.utils import visualization_utils as viz_utils

import inspect
import sys
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
from exercise_ws.src.object_detection.include.object_detection.model import Wrapper
from exercise_ws.src.object_detection.include.object_detection.dataset import Dataset

# To prevent the process to use all the VRAM if not necessary
# Also, allow to use a RTX NVIDIA GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

%matplotlib inline

## Utilities

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None,
                    class_labels=True,
                    score_labels=True,
                    min_score_thresh=0.5):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=min_score_thresh,
      line_thickness=1,
      skip_scores=not score_labels,
      skip_labels=not class_labels)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)


# Test .tflite model

## Load .tflite model

In [ ]:
model = Wrapper(None)

## Generate prediction from each frame

In [ ]:
test_image_dir = './test/originals'
gif_frames_dir = './test/frames'
test_images_np = []

dataset = Dataset()

for i in range(1, 100):
  image_path = os.path.join(test_image_dir, str(i) + '.npz')
  test_images_np.append(np.expand_dims(np.load(image_path)["arr_0"], axis=0))

# Note that the first frame will trigger tracing of the tf.function, which will
# take some time, after which inference should be fast.

label_id_offset = 1
for i in range(len(test_images_np)):
  input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
  boxes, classes, scores = model.predict(input_tensor)
  if i == 1:
    print(boxes, classes, scores)

  plot_detections(
      test_images_np[i][0],
      boxes,
      classes.astype(np.uint32) + dataset.label_id_offset,
      scores,
      dataset.category_index, 
      figsize=(15, 20), 
      image_name=gif_frames_dir + "/gif_frame_" + ('%02d' % i) + ".jpg",
      min_score_thresh=0.3
  )

## Assemble each processed frames into a GIF

In [ ]:
imageio.plugins.freeimage.download()

anim_file = 'duckies_test.gif'

filenames = glob.glob(gif_frames_dir + '/gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []
for filename in filenames:
  image = imageio.imread(filename)
  images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=5)

display(IPyImage(open(anim_file, 'rb').read()))

# Check the dataset validity

## Process a batch of files

In [ ]:
batch_size = 32
check_frames_dir = './check/frames'

dataset = Dataset()
ragged_batches = dataset.frames.apply(
  tf.data.experimental.dense_to_ragged_batch(batch_size=batch_size)
)

for batch in ragged_batches:
    images, gt_boxes, gt_classes = batch
    
    images_np = images.numpy().astype(np.uint8)
    boxes_np = gt_boxes.numpy()
    classes_np = gt_classes.numpy()

    # Decode one-hot encoding
    for i in range(classes_np.shape[0]):
        classes_np[i] = np.argmax(classes_np[i], axis=1)
    
    for i in range(batch_size):
        if boxes_np[i].shape[0] > 0:
          plot_detections(
            images_np[i],
            boxes_np[i],
            classes_np[i].astype(np.uint8) + label_id_offset,
            np.ones(boxes_np[i].shape[0]),
            dataset.category_index,
            figsize=(15, 20),
            image_name=check_frames_dir + "/gif_frame_" + ('%02d' % i) + ".jpg",
            class_labels=True,
            score_labels=False
          )
    break

## Assemble each processed frames into a GIF

In [ ]:
imageio.plugins.freeimage.download()

anim_file = 'duckies_check_db.gif'

filenames = glob.glob(check_frames_dir + '/gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []
for filename in filenames:
  image = imageio.imread(filename)
  images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=0.5)

display(IPyImage(open(anim_file, 'rb').read()))